In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from pandas.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.optimize import minimize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [2]:
train_proteins_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print('proteins :', train_proteins_df.shape)
print('proteins unique visit_ids:', train_proteins_df.visit_id.nunique())
print('proteins unique patient_ids:', train_proteins_df.patient_id.nunique())
print(train_proteins_df.columns)

train_peptides_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
print('')
print('peptides :', train_peptides_df.shape)
print('peptides unique visit_ids:', train_peptides_df.visit_id.nunique())
print('peptides unique patient_ids:', train_peptides_df.patient_id.nunique())
print(train_peptides_df.columns)

train_clinical_data_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
print('')
print('clinical :', train_clinical_data_df.shape)
print('clinical unique visit_ids:', train_clinical_data_df.visit_id.nunique())
print(train_clinical_data_df.columns)

"""supp_clinical_data_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
print('')
print('supplemental :', supp_clinical_data_df.shape)
print('supplemental unique visit_ids:', supp_clinical_data_df.visit_id.nunique())
print(supp_clinical_data_df.columns)"""

proteins : (232741, 5)
proteins unique visit_ids: 1113
proteins unique patient_ids: 248
Index(['visit_id', 'visit_month', 'patient_id', 'UniProt', 'NPX'], dtype='object')

peptides : (981834, 6)
peptides unique visit_ids: 1113
peptides unique patient_ids: 248
Index(['visit_id', 'visit_month', 'patient_id', 'UniProt', 'Peptide',
       'PeptideAbundance'],
      dtype='object')

clinical : (2615, 8)
clinical unique visit_ids: 2615
Index(['visit_id', 'patient_id', 'visit_month', 'updrs_1', 'updrs_2',
       'updrs_3', 'updrs_4', 'upd23b_clinical_state_on_medication'],
      dtype='object')


"supp_clinical_data_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')\nprint('')\nprint('supplemental :', supp_clinical_data_df.shape)\nprint('supplemental unique visit_ids:', supp_clinical_data_df.visit_id.nunique())\nprint(supp_clinical_data_df.columns)"

In [3]:
clinical_data = train_clinical_data_df
clinical_data.drop(columns=['upd23b_clinical_state_on_medication'], inplace=True)

"""pd.merge(supp_clinical_data_df, train_clinical_data_df, on=['patient_id','visit_id','visit_month'], how='outer')

for i in [1,2,3,4]:
    clinical_data['updrs_'+str(i)] = np.where(clinical_data['updrs_'+str(i)+'_x'].isna(), clinical_data['updrs_'+str(i)+'_y'], clinical_data['updrs_'+str(i)+'_x'])
    clinical_data.drop(columns=['updrs_'+str(i)+'_x', 'updrs_'+str(i)+'_y'], inplace=True)
    
clinical_data.drop(columns=['upd23b_clinical_state_on_medication_x', 'upd23b_clinical_state_on_medication_y'], inplace=True)"""

print(clinical_data.shape)
print(clinical_data.visit_id.nunique())

pro_pep_join = pd.merge(train_proteins_df, train_peptides_df, on=['patient_id','visit_id','visit_month','UniProt'], how='outer')

print(pro_pep_join.shape)
print(pro_pep_join.visit_id.nunique())

del train_proteins_df, train_peptides_df

(2615, 7)
2615
(981834, 7)
1113


In [4]:
def remove_unimod(peptide_seqn):
    return re.sub('[(UniMod_)0-9]+','',peptide_seqn)

#print(remove_unimod('KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK'))

pro_pep_join['Peptide'] = pro_pep_join['Peptide'].apply(remove_unimod)
#pro_pep_join.head(10)

"""def amino_acid_count(amino_seqn, amino_acid):
    return amino_seqn.count(amino_acid)"""

def amino_acid_abundance(amino_seqn, amino_acid, peptide_abundance, amino_acid_abundance):
    return round(amino_seqn.count(amino_acid) * ((peptide_abundance * amino_acid_abundance)/100),5)

vect_func = np.vectorize(amino_acid_abundance)

pro_pep_join['I_abundance'] = vect_func(pro_pep_join['Peptide'], 'I', pro_pep_join['PeptideAbundance'], 6.73)
pro_pep_join['V_abundance'] = vect_func(pro_pep_join['Peptide'], 'V', pro_pep_join['PeptideAbundance'], 5.49)
pro_pep_join['F_abundance'] = vect_func(pro_pep_join['Peptide'], 'F', pro_pep_join['PeptideAbundance'], 3.87)
pro_pep_join['Y_abundance'] = vect_func(pro_pep_join['Peptide'], 'Y', pro_pep_join['PeptideAbundance'], 2.91)
pro_pep_join['W_abundance'] = vect_func(pro_pep_join['Peptide'], 'W', pro_pep_join['PeptideAbundance'], 1.25)
pro_pep_join['A_abundance'] = vect_func(pro_pep_join['Peptide'], 'A', pro_pep_join['PeptideAbundance'], 8.76)
pro_pep_join['L_abundance'] = vect_func(pro_pep_join['Peptide'], 'L', pro_pep_join['PeptideAbundance'], 9.68)
    
#pro_pep_join.head(10)

pro_pep_join_agg = pro_pep_join.groupby(['patient_id','visit_id','visit_month']).agg(F=('F_abundance', 'sum'),Y=('Y_abundance', 'sum'),W=('W_abundance', 'sum')\
                                                                                    ,A=('A_abundance', 'sum'),L=('L_abundance', 'sum'),V=('V_abundance', 'sum')\
                                                                                    ,I=('I_abundance', 'sum')).reset_index(level=['patient_id','visit_id','visit_month'])

pro_pep_join_agg.head(10)

,patient_id,visit_id,visit_month,F,Y,W,A,L,V,I
0,55,55_0,0,1.187186e+07,7.520846e+06,478382.42875,5.734561e+07,7.342820e+07,5.544380e+07,8.991024e+06
1,55,55_12,12,1.249837e+07,7.993317e+06,484364.63337,5.792294e+07,7.600235e+07,5.324986e+07,9.488818e+06
2,55,55_36,36,1.319476e+07,8.373198e+06,533001.00401,6.335465e+07,8.060006e+07,5.719920e+07,8.898225e+06
3,55,55_6,6,1.134211e+07,7.076986e+06,450325.41649,5.478653e+07,6.809448e+07,4.880191e+07,8.293410e+06
4,942,942_12,12,1.017770e+07,6.625098e+06,445507.01004,4.559631e+07,6.117932e+07,4.064128e+07,6.966456e+06
5,942,942_24,24,9.903217e+06,6.586329e+06,452366.38213,5.055081e+07,6.051721e+07,4.143208e+07,7.087011e+06
6,942,942_48,48,1.077650e+07,6.268919e+06,449311.39699,5.409210e+07,6.229291e+07,4.287919e+07,6.806131e+06
7,942,942_6,6,9.460291e+06,6.086457e+06,438816.94926,4.581346e+07,5.843574e+07,3.874751e+07,7.189919e+06
8,1517,1517_0,0,9.868730e+06,6.870025e+06,402572.45311,5.120800e+07,6.078189e+07,3.840256e+07,7.175566e+06
9,1517,1517_24,24,9.686343e+06,6.386043e+06,398692.42975,4.693729e+07,5.986200e+07,3.941939e+07,7.466161e+06


In [5]:
full_training_data = pd.merge(pro_pep_join_agg, clinical_data, on=['patient_id','visit_id','visit_month'], how='outer')

print(full_training_data.shape)
print(full_training_data.visit_id.nunique())

del pro_pep_join, clinical_data
gc.collect()

columns_to_remove_nan_from = ['updrs_1','updrs_2','updrs_3','updrs_4','F','Y','W','L','A','I','V']

for i in columns_to_remove_nan_from:
    if i in(['F','Y','W','L','A','I','V']):
        full_training_data[i].fillna(0, inplace=True)
    else:
        full_training_data[i].fillna(full_training_data[i].median().round(decimals = 0), inplace=True)
        
full_training_data = full_training_data.drop_duplicates()

full_training_data_sorted = full_training_data.sort_values(by=['patient_id','visit_month'],  ascending=True)

full_training_data_sorted['visit_month_lag'] = full_training_data_sorted.groupby(['patient_id'])['visit_month'].shift(1)
full_training_data_sorted['visit_month_lag'].fillna(-1, inplace=True)

full_training_data_sorted['visit_month_lead'] = full_training_data_sorted.groupby(['patient_id'])['visit_month'].shift(-1)
full_training_data_sorted['visit_month_lead'].fillna(-1, inplace=True)

"""full_training_data_sorted['visit_month_lead'] = full_training_data_sorted.groupby(['patient_id'])['visit_month'].shift(-1)
full_training_data_sorted['visit_month_lead'].fillna(-1, inplace=True)"""

full_training_data_sorted = full_training_data_sorted.reset_index()
full_training_data_sorted = full_training_data_sorted.drop(columns=['patient_id','index'])

full_training_data_sorted = full_training_data_sorted[~full_training_data_sorted.visit_month.isin([3,5,9])]

del full_training_data
gc.collect()
full_training_data_sorted.head(10)

(2660, 14)
2660


,visit_id,visit_month,F,Y,W,A,L,V,I,updrs_1,updrs_2,updrs_3,updrs_4,visit_month_lag,visit_month_lead
0,55_0,0,1.187186e+07,7.520846e+06,478382.42875,5.734561e+07,7.342820e+07,5.544380e+07,8.991024e+06,10.0,6.0,15.0,0.0,-1.0,3.0
2,55_6,6,1.134211e+07,7.076986e+06,450325.41649,5.478653e+07,6.809448e+07,4.880191e+07,8.293410e+06,8.0,10.0,34.0,0.0,3.0,9.0
4,55_12,12,1.249837e+07,7.993317e+06,484364.63337,5.792294e+07,7.600235e+07,5.324986e+07,9.488818e+06,10.0,10.0,41.0,0.0,9.0,18.0
5,55_18,18,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.0,13.0,38.0,0.0,12.0,24.0
6,55_24,24,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,16.0,9.0,49.0,0.0,18.0,30.0
7,55_30,30,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,14.0,13.0,49.0,0.0,24.0,36.0
8,55_36,36,1.319476e+07,8.373198e+06,533001.00401,6.335465e+07,8.060006e+07,5.719920e+07,8.898225e+06,17.0,18.0,51.0,0.0,30.0,42.0
9,55_42,42,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,12.0,20.0,41.0,0.0,36.0,48.0
10,55_48,48,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,17.0,16.0,52.0,0.0,42.0,54.0
11,55_54,54,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,12.0,18.0,51.0,0.0,48.0,60.0


In [6]:
full_training_data_sorted[['updrs_1','updrs_2','updrs_3','updrs_4','visit_month','visit_month_lag','visit_month_lead','F','Y','W','A','L','I','V']].corr()

,updrs_1,updrs_2,updrs_3,updrs_4,visit_month,visit_month_lag,visit_month_lead,F,Y,W,A,L,I,V
updrs_1,1.000000,0.665443,0.445788,0.448181,0.110016,0.131821,0.033073,-0.116715,-0.123971,-0.111039,-0.119832,-0.121241,-0.114847,-0.117724
updrs_2,0.665443,1.000000,0.744440,0.428652,0.147401,0.182291,0.047935,-0.134807,-0.143379,-0.125525,-0.138157,-0.139206,-0.137162,-0.138339
updrs_3,0.445788,0.744440,1.000000,0.309066,0.148829,0.190715,0.061265,-0.120918,-0.125307,-0.114422,-0.124197,-0.122286,-0.125536,-0.123277
updrs_4,0.448181,0.428652,0.309066,1.000000,0.168949,0.180922,0.072366,-0.083077,-0.086754,-0.080544,-0.084875,-0.084935,-0.080254,-0.081454
visit_month,0.110016,0.147401,0.148829,0.168949,1.000000,0.990561,0.521918,-0.248694,-0.246064,-0.238046,-0.248207,-0.250367,-0.245411,-0.251922
visit_month_lag,0.131821,0.182291,0.190715,0.180922,0.990561,1.000000,0.509365,-0.248835,-0.246074,-0.237210,-0.248195,-0.250472,-0.245525,-0.252068
visit_month_lead,0.033073,0.047935,0.061265,0.072366,0.521918,0.509365,1.000000,-0.131660,-0.130446,-0.123016,-0.131800,-0.134008,-0.128798,-0.134500
F,-0.116715,-0.134807,-0.120918,-0.083077,-0.248694,-0.248835,-0.131660,1.000000,0.992062,0.969748,0.995481,0.996843,0.990655,0.992391
Y,-0.123971,-0.143379,-0.125307,-0.086754,-0.246064,-0.246074,-0.130446,0.992062,1.000000,0.969777,0.994633,0.995605,0.989512,0.986259
W,-0.111039,-0.125525,-0.114422,-0.080544,-0.238046,-0.237210,-0.123016,0.969748,0.969777,1.000000,0.978485,0.968709,0.973324,0.965117


In [7]:
full_training_data_sorted.isna().sum()

visit_id            0
visit_month         0
F                   0
Y                   0
W                   0
A                   0
L                   0
V                   0
I                   0
updrs_1             0
updrs_2             0
updrs_3             0
updrs_4             0
visit_month_lag     0
visit_month_lead    0
dtype: int64

In [8]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

In [9]:
ridge_model_dict = {}
#model_dict = {}
list_of_updrs_tests = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

def train_n_predict_updrs_3(full_training_data_sorted):
    temp = full_training_data_sorted
    X = temp[['visit_month','visit_month_lag']]
    print(X.shape)
    y = temp['updrs_3']
    print(y.shape)
    poly = PolynomialFeatures(degree = 7)
    X_poly = poly.fit_transform(X)
    poly.fit(X_poly, y)
    print(X_poly.shape)
    #trained = LinearRegression().fit(X_poly, y)
    #trained = RidgeCV(alphas=[0.001,0.0001,0.01], gcv_mode='svd', alpha_per_target=True).fit(X, y)
    #trained = Lasso(alpha=1.03, max_iter=4500).fit(X, y)
    trained = Ridge(alpha=1.01, solver="cholesky").fit(X_poly, y)
    #trained = SGDRegressor(alpha=0.011, learning_rate='optimal', early_stopping=True, validation_fraction=0.1).fit(X, y)
    #trained = LinearRegression().fit(X, y)
    #trained = RandomForestRegressor().fit(X, y)
    #random_forest_model_dict['updrs_3'] = trained
    #poly_model_dict['updrs_3'] = trained
    #model_dict['updrs_3'] = trained
    ridge_model_dict['updrs_3'] = trained
    #sgd_model_dict['updrs_3'] = trained
    #lasso_model_dict['updrs_3'] = trained

    for m in [0, 6, 12, 24]:
        temp['predicted_updrs_3_plus_'+str(m)+'_months'] = np.ceil(ridge_model_dict['updrs_3'].predict(X_poly))
        temp['predicted_updrs_3_plus_'+str(m)+'_months'] = temp['predicted_updrs_3_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
    
    return temp

In [10]:
def train_n_predict_updrs_2(temp):
    for m in [0, 6, 12, 24]:
        X = temp[['visit_month','visit_month_lag','predicted_updrs_3_plus_'+str(m)+'_months']]
        print(X.shape)
        y = temp['updrs_2']
        print(y.shape)
        poly = PolynomialFeatures(degree = 7)
        X_poly = poly.fit_transform(X)
        poly.fit(X_poly, y)
        print(X_poly.shape)
        #trained = LinearRegression().fit(X_poly, y)
        #trained = RidgeCV(alphas=[0.001,0.0001,0.01], gcv_mode='svd', alpha_per_target=True).fit(X, y)
        trained = Ridge(alpha=1.01, solver="cholesky").fit(X_poly, y)
        #trained = Lasso(alpha=1.03, max_iter=4500).fit(X, y)
        #trained = SGDRegressor(alpha=0.011, learning_rate='optimal', early_stopping=True, validation_fraction=0.1).fit(X, y)
        #trained = LinearRegression().fit(X, y)
        #trained = RandomForestRegressor().fit(X, y)
        #random_forest_model_dict['updrs_2'] = trained
        #poly_model_dict['updrs_2'] = trained
        #model_dict['updrs_2'] = trained
        ridge_model_dict['updrs_2_plus_'+str(m)+'_months'] = trained
        #sgd_model_dict['updrs_2'] = trained
        #lasso_model_dict['updrs_2'] = trained
    
    
        temp['predicted_updrs_2_plus_'+str(m)+'_months'] = np.ceil(ridge_model_dict['updrs_2_plus_'+str(m)+'_months'].predict(X_poly))
        temp['predicted_updrs_2_plus_'+str(m)+'_months'] = temp['predicted_updrs_2_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
    
    return temp

In [11]:
def train_n_predict_updrs_1(temp):
    #X = temp[['visit_month','visit_month_lag']]
    for m in [0, 6, 12, 24]:
        X = temp[['visit_month','visit_month_lag','predicted_updrs_2_plus_'+str(m)+'_months','predicted_updrs_3_plus_'+str(m)+'_months']]
        print(X.shape)
        y = temp['updrs_1']
        print(y.shape)
        poly = PolynomialFeatures(degree = 11)
        X_poly = poly.fit_transform(X)
        poly.fit(X_poly, y)
        print(X_poly.shape)
        #trained = LinearRegression().fit(X_poly, y)
        #trained = LinearRegression().fit(X, y)
        #trained = RidgeCV(alphas=[0.001,0.0001,0.01], gcv_mode='svd', alpha_per_target=True).fit(X, y)
        trained = Ridge(alpha=1.01, solver="cholesky").fit(X_poly, y)
        #trained = Lasso(alpha=1.03, max_iter=4500).fit(X, y)
        #trained = SGDRegressor(alpha=0.011, learning_rate='optimal', early_stopping=True, validation_fraction=0.1).fit(X, y)
        #trained = RandomForestRegressor().fit(X, y)
        #random_forest_model_dict['updrs_1'] = trained
        #poly_model_dict['updrs_1'] = trained
        #model_dict['updrs_1'] = trained
        ridge_model_dict['updrs_1_plus_'+str(m)+'_months'] = trained
        #sgd_model_dict['updrs_1'] = trained
        #lasso_model_dict['updrs_1'] = trained

        temp['predicted_updrs_1_plus_'+str(m)+'_months'] = np.ceil(ridge_model_dict['updrs_1_plus_'+str(m)+'_months'].predict(X_poly))
        temp['predicted_updrs_1_plus_'+str(m)+'_months'] = temp['predicted_updrs_1_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
        #print(round(ridge_model_dict['updrs_1'].score(X, y),5))
    
    return temp

In [12]:
def train_n_predict_updrs_4(temp):
    for m in [0, 6, 12, 24]:
        X = temp[['visit_month','visit_month_lag','visit_month_lead','predicted_updrs_1_plus_'+str(m)+'_months','predicted_updrs_2_plus_'+str(m)+'_months'\
                  ,'predicted_updrs_3_plus_'+str(m)+'_months']]
        print(X.shape)
        y = temp['updrs_4']
        print(y.shape)
        poly = PolynomialFeatures(degree = 7)
        X_poly = poly.fit_transform(X)
        poly.fit(X_poly, y)
        print(X_poly.shape)
        #trained = LinearRegression().fit(X_poly, y)
        #trained = RidgeCV(alphas=[0.001,0.0001,0.01], gcv_mode='svd', alpha_per_target=True).fit(X, y)
        trained = Ridge(alpha=1.01).fit(X_poly, y)
        #trained = Lasso(alpha=1.03, max_iter=4500).fit(X, y)
        #trained = SGDRegressor(alpha=0.051, learning_rate='optimal', early_stopping=True, validation_fraction=0.1).fit(X, y)
        #trained = LinearRegression().fit(X, y)
        #trained = RandomForestRegressor().fit(X, y)
        #random_forest_model_dict['updrs_4'] = trained
        #poly_model_dict['updrs_4'] = trained
        #model_dict['updrs_4'] = trained
        ridge_model_dict['updrs_4_plus_'+str(m)+'_months'] = trained
        #sgd_model_dict['updrs_4'] = trained
        #lasso_model_dict['updrs_4'] = trained

        temp['predicted_updrs_4_plus_'+str(m)+'_months'] = np.ceil(ridge_model_dict['updrs_4_plus_'+str(m)+'_months'].predict(X_poly))
        temp['predicted_updrs_4_plus_'+str(m)+'_months'] = temp['predicted_updrs_4_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
    
    return temp

In [13]:
temp = train_n_predict_updrs_4(
    train_n_predict_updrs_1(
    train_n_predict_updrs_2(
    train_n_predict_updrs_3(full_training_data_sorted)
)
)
)

"""for u in list_of_updrs_tests:
    
    #y_true = full_training_data_sorted[u]
    y_true = temp[u]
    y_pred = temp['predicted_'+u]
    
    print('SMAPE + 1 for', u, ':', smape_plus_1(np.array(y_true), np.array(y_pred)))"""

(2444, 2)
(2444,)
(2444, 36)
(2444, 3)
(2444,)
(2444, 120)
(2444, 3)
(2444,)
(2444, 120)
(2444, 3)
(2444,)
(2444, 120)
(2444, 3)
(2444,)
(2444, 120)
(2444, 4)
(2444,)
(2444, 1365)
(2444, 4)
(2444,)
(2444, 1365)
(2444, 4)
(2444,)
(2444, 1365)
(2444, 4)
(2444,)
(2444, 1365)
(2444, 6)
(2444,)
(2444, 1716)
(2444, 6)
(2444,)
(2444, 1716)
(2444, 6)
(2444,)
(2444, 1716)
(2444, 6)
(2444,)
(2444, 1716)


"for u in list_of_updrs_tests:\n    \n    #y_true = full_training_data_sorted[u]\n    y_true = temp[u]\n    y_pred = temp['predicted_'+u]\n    \n    print('SMAPE + 1 for', u, ':', smape_plus_1(np.array(y_true), np.array(y_pred)))"

In [14]:
"""pred_vs_true_melt = pd.melt(temp, id_vars=['visit_id'], value_vars=['updrs_1','updrs_2','updrs_3','updrs_4','predicted_updrs_1','predicted_updrs_2','predicted_updrs_3'\
                                                                    ,'predicted_updrs_4'])

pred_vs_true_melt.head(10)

y_true_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['updrs_1','updrs_2','updrs_3','updrs_4'])]['value']
y_pred_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['predicted_updrs_1','predicted_updrs_2','predicted_updrs_3','predicted_updrs_4'])]['value']

print('SMAPE + 1 on overall train data :', smape_plus_1(np.array(y_true_overall), np.array(y_pred_overall)))"""

"pred_vs_true_melt = pd.melt(temp, id_vars=['visit_id'], value_vars=['updrs_1','updrs_2','updrs_3','updrs_4','predicted_updrs_1','predicted_updrs_2','predicted_updrs_3'                                                                    ,'predicted_updrs_4'])\n\npred_vs_true_melt.head(10)\n\ny_true_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['updrs_1','updrs_2','updrs_3','updrs_4'])]['value']\ny_pred_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['predicted_updrs_1','predicted_updrs_2','predicted_updrs_3','predicted_updrs_4'])]['value']\n\nprint('SMAPE + 1 on overall train data :', smape_plus_1(np.array(y_true_overall), np.array(y_pred_overall)))"

In [15]:
def get_full_test_data(df_test, df_proteins=pd.DataFrame(), df_peptides=pd.DataFrame()):
    
    if  (df_proteins.shape[0] == 0) & (df_peptides.shape[0] == 0):
        
        print('only the test dataframe has data, proteins and peptides info absent')
        
        full_test_data = df_test[['patient_id','visit_id','visit_month']]
        full_test_data = full_test_data.drop_duplicates()
        full_test_data_sorted = full_test_data.sort_values(by=['patient_id','visit_month'],  ascending=True)
        full_test_data_sorted['visit_month_lag'] = full_test_data_sorted.groupby(['patient_id'])['visit_month'].shift(1)
        full_test_data_sorted['visit_month_lag'].fillna(-1, inplace=True)
        
        full_test_data_sorted = full_test_data_sorted.drop_duplicates()
        full_test_data_sorted = full_test_data_sorted[['visit_id','visit_month','visit_month_lag']]

        full_test_data_sorted = full_test_data_sorted.reset_index()
        full_test_data_sorted = full_test_data_sorted.drop(columns=['index'])
        
        return full_test_data_sorted
    
    elif (df_proteins.shape[0] == 0) & (df_peptides.shape[0] != 0):
        
        print('no proteins only peptides')
        print('df_test shape and number of unique visit_ids:', df_test.shape, df_test.visit_id.nunique())
        print('df_peptides shape and number of unique visit_ids:', df_peptides.shape, df_peptides.visit_id.nunique())
        df_test = df_test[['patient_id','visit_id','visit_month']]
        df_test = df_test.drop_duplicates()
        
        df_peptides['Peptide'] = df_peptides['Peptide'].apply(remove_unimod)
        """df_peptides['W_count'] = vect_func(df_peptides['Peptide'], 'W')
        df_peptides['Y_count'] = vect_func(df_peptides['Peptide'], 'Y')
        df_peptides['F_count'] = vect_func(df_peptides['Peptide'], 'F')
        df_peptides['L_count'] = vect_func(df_peptides['Peptide'], 'L')
        df_peptides['A_count'] = vect_func(df_peptides['Peptide'], 'A')"""
        
        df_peptides['I_abundance'] = vect_func(df_peptides['Peptide'], 'I', df_peptides['PeptideAbundance'], 6.73)
        df_peptides['V_abundance'] = vect_func(df_peptides['Peptide'], 'V', df_peptides['PeptideAbundance'], 5.49)
        df_peptides['F_abundance'] = vect_func(df_peptides['Peptide'], 'F', df_peptides['PeptideAbundance'], 3.87)
        df_peptides['Y_abundance'] = vect_func(df_peptides['Peptide'], 'Y', df_peptides['PeptideAbundance'], 2.91)
        df_peptides['W_abundance'] = vect_func(df_peptides['Peptide'], 'W', df_peptides['PeptideAbundance'], 1.25)
        df_peptides['A_abundance'] = vect_func(df_peptides['Peptide'], 'A', df_peptides['PeptideAbundance'], 8.76)
        df_peptides['L_abundance'] = vect_func(df_peptides['Peptide'], 'L', df_peptides['PeptideAbundance'], 9.68)
    
        

        df_peptides_agg = df_peptides.groupby(['patient_id','visit_id','visit_month']).agg(F=('F_abundance', 'sum'),Y=('Y_abundance', 'sum'),W=('W_abundance', 'sum')\
                                                                                           ,A=('A_abundance', 'sum'),L=('L_abundance', 'sum'),V=('V_abundance', 'sum')\
                                                                                           ,I=('I_abundance', 'sum')).reset_index(level=['patient_id','visit_id','visit_month'])
        

        """df_peptides_agg = df_peptides.groupby(['patient_id','visit_id','visit_month']).agg(W=('W_count', 'sum'),Y=('Y_count', 'sum'),F=('F_count', 'sum')\
                                                                                          ,A=('A_count', 'sum'),L=('L_count', 'sum'))\
        .reset_index(level=['patient_id','visit_id','visit_month'])"""
        
        df_peptides = pd.merge(df_test, df_peptides_agg, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='left')
        df_peptides = df_peptides[['patient_id','visit_id','visit_month','F','Y','W','A','L','V','I']]
        
        columns_to_remove_nan_from = ['F','Y','W','L','A','I','V']

        for i in columns_to_remove_nan_from:
            df_peptides[i].fillna(0, inplace=True)
        
        df_peptides = df_peptides.drop_duplicates()
        
        df_peptides_sorted = df_peptides.sort_values(by=['patient_id','visit_month'],  ascending=True)
        df_peptides_sorted['visit_month_lag'] = df_peptides_sorted.groupby(['patient_id'])['visit_month'].shift(1)
        df_peptides_sorted['visit_month_lag'].fillna(-1, inplace=True)
        
        df_peptides_sorted['visit_month_lead'] = df_peptides_sorted.groupby(['patient_id'])['visit_month'].shift(-1)
        df_peptides_sorted['visit_month_lead'].fillna(-1, inplace=True)

        """
        full_test_data = pd.DataFrame()
        for i in [0,6,12,24]:
            temp = df_peptides[df_peptides['visit_month']==i]
            full_test_data = full_test_data.append(temp)
        
        del df_proteins, df_peptides
        """
        full_test_data = df_peptides_sorted
        del df_peptides_sorted
        print('full_test_data shape and number of unique visit_ids:', full_test_data.shape, full_test_data.visit_id.nunique())
        
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month', 'visit_month_lag','visit_month_lead','F','Y','W','A','L','V','I']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        print(full_test_data.head(10))
        
        return full_test_data
            
    elif (df_peptides.shape[0] == 0) & (df_proteins.shape[0] != 0):
        
        print('no peptides only proteins')
        print('df_test shape and number of unique visit_ids:', df_test.shape, df_test.visit_id.nunique())
        print('df_proteins shape and number of unique visit_ids:', df_proteins.shape, df_proteins.visit_id.nunique())
        df_test = df_test[['patient_id','visit_id','visit_month']]
        df_test = df_test.drop_duplicates()
        
        df_proteins = pd.merge(df_test, df_proteins, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='left')
        df_proteins = df_proteins[['patient_id','visit_id','visit_month']]
        df_proteins = df_proteins.drop_duplicates()
        
        df_proteins_sorted = df_proteins.sort_values(by=['patient_id','visit_month'],  ascending=True)
        df_proteins_sorted['visit_month_lag'] = df_proteins_sorted.groupby(['patient_id'])['visit_month'].shift(1)
        df_proteins_sorted['visit_month_lag'].fillna(-1, inplace=True)

        """
        full_test_data = pd.DataFrame()
        for i in [0,6,12,24]:
            temp = df_proteins[df_proteins['visit_month']==i]
            full_test_data = full_test_data.append(temp)
        """
        full_test_data = df_proteins_sorted
        print('full_test_data shape and number of unique visit_ids:', full_test_data.shape, full_test_data.visit_id.nunique())
        
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month', 'visit_month_lag']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        return full_test_data
        
    else:
        
        print('both proteins and peptides are present')
        
        df_test = df_test[['patient_id','visit_id','visit_month']]
        df_test = df_test.drop_duplicates()
        
        pro_pep_join = pd.merge(df_proteins, df_peptides, on=['patient_id','visit_id','visit_month','UniProt'], suffixes=['_left','_right'], how='outer')
        pro_pep_join = pro_pep_join[['patient_id','visit_id','visit_month']]
        pro_pep_join = pro_pep_join.drop_duplicates()
        #del df_proteins, df_peptides
        print('df_proteins shape and number of unique visit_ids:', df_proteins.shape, df_proteins.visit_id.nunique())
        print('df_peptides shape and number of unique visit_ids:', df_peptides.shape, df_peptides.visit_id.nunique())
        print('pro_pep_join shape and number of unique visit_ids:', pro_pep_join.shape, pro_pep_join.visit_id.nunique())
        
        pro_pep_join_test = pd.merge(df_test, pro_pep_join, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='left')
        
        pro_pep_join_test_sorted = pro_pep_join_test.sort_values(by=['patient_id','visit_month'],  ascending=True)
        pro_pep_join_test_sorted['visit_month_lag'] = pro_pep_join_test_sorted.groupby(['patient_id'])['visit_month'].shift(1)
        pro_pep_join_test_sorted['visit_month_lag'].fillna(-1, inplace=True)

        """
        full_test_data = pd.DataFrame()

        for i in [0,6,12,24]:
            temp = pro_pep_join_test[pro_pep_join_test['visit_month']==i]
            full_test_data = full_test_data.append(temp)
        """
        full_test_data = pro_pep_join_test_sorted 
        print('full_test_data shape and number of unique visit_ids:', full_test_data.shape, full_test_data.visit_id.nunique())

        full_test_data = full_test_data.drop_duplicates()
        full_test_data = full_test_data[['visit_id','visit_month', 'visit_month_lag']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        return full_test_data

In [16]:
ridge_model_dict

{'updrs_3': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_2_plus_0_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_2_plus_6_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_2_plus_12_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_2_plus_24_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_1_plus_0_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_1_plus_6_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_1_plus_12_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_1_plus_24_months': Ridge(alpha=1.01, solver='cholesky'),
 'updrs_4_plus_0_months': Ridge(alpha=1.01),
 'updrs_4_plus_6_months': Ridge(alpha=1.01),
 'updrs_4_plus_12_months': Ridge(alpha=1.01),
 'updrs_4_plus_24_months': Ridge(alpha=1.01)}

In [17]:
def get_predictions_v4(model_dict, df_test, df_proteins=pd.DataFrame(), df_peptides=pd.DataFrame()):
    
    list_of_updrs_tests = ['updrs_1','updrs_2','updrs_3','updrs_4']
    result = pd.DataFrame()
    
    full_test_data = get_full_test_data(df_test=df_test, df_proteins=df_proteins, df_peptides=df_peptides)
    
    if full_test_data.shape[0] != 0:
        """df_prepared = sklearn_pipeline.transform(full_test_data)
        print(df_prepared.shape)"""

        ##for u in list_of_updrs_tests:
        
        X = full_test_data[['visit_month','visit_month_lag']]
        poly = PolynomialFeatures(degree = 7)
        X_poly = poly.fit_transform(X)
        
        """full_test_data['predicted_updrs_3'] = np.ceil(model_dict['updrs_3'].predict(X_poly))
        full_test_data['predicted_updrs_3'] = full_test_data['predicted_updrs_3'].apply(lambda x: 0 if x < 0 else x)"""
        
        for m in [0, 6, 12, 24]:
            full_test_data['predicted_updrs_3_plus_'+str(m)+'_months'] = np.ceil(model_dict['updrs_3'].predict(X_poly))
            full_test_data['predicted_updrs_3_plus_'+str(m)+'_months'] = full_test_data['predicted_updrs_3_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
        
        for m in [0, 6, 12, 24]:
            X = full_test_data[['visit_month','visit_month_lag','predicted_updrs_3_plus_'+str(m)+'_months']]
            poly = PolynomialFeatures(degree = 7)
            X_poly = poly.fit_transform(X)
            
            full_test_data['predicted_updrs_2_plus_'+str(m)+'_months'] = np.ceil(model_dict['updrs_2_plus_'+str(m)+'_months'].predict(X_poly))
            full_test_data['predicted_updrs_2_plus_'+str(m)+'_months'] = full_test_data['predicted_updrs_2_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
        
        """full_test_data['predicted_updrs_2'] = np.ceil(model_dict['updrs_2'].predict(X_poly))
        full_test_data['predicted_updrs_2'] = full_test_data['predicted_updrs_2'].apply(lambda x: 0 if x < 0 else x)"""
        
        for m in [0, 6, 12, 24]:
            X = full_test_data[['visit_month','visit_month_lag','predicted_updrs_2_plus_'+str(m)+'_months','predicted_updrs_3_plus_'+str(m)+'_months']]
            poly = PolynomialFeatures(degree = 11)
            X_poly = poly.fit_transform(X)
            
            full_test_data['predicted_updrs_1_plus_'+str(m)+'_months'] = np.ceil(model_dict['updrs_1_plus_'+str(m)+'_months'].predict(X_poly))
            full_test_data['predicted_updrs_1_plus_'+str(m)+'_months'] = full_test_data['predicted_updrs_1_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
            
            
        """full_test_data['predicted_updrs_1'] = np.ceil(model_dict['updrs_1'].predict(X_poly))
        full_test_data['predicted_updrs_1'] = full_test_data['predicted_updrs_1'].apply(lambda x: 0 if x < 0 else x)"""
        
        for m in [0, 6, 12, 24]:
            X = full_test_data[['visit_month','visit_month_lag','visit_month_lead','predicted_updrs_1_plus_'+str(m)+'_months','predicted_updrs_2_plus_'+str(m)+'_months'\
                                ,'predicted_updrs_3_plus_'+str(m)+'_months']]
            poly = PolynomialFeatures(degree = 7)
            X_poly = poly.fit_transform(X)
            
            full_test_data['predicted_updrs_4_plus_'+str(m)+'_months'] = np.ceil(model_dict['updrs_4_plus_'+str(m)+'_months'].predict(X_poly))
            full_test_data['predicted_updrs_4_plus_'+str(m)+'_months'] = full_test_data['predicted_updrs_4_plus_'+str(m)+'_months'].apply(lambda x: 0 if x < 0 else x)
            
        """full_test_data['predicted_updrs_4'] = np.ceil(model_dict['updrs_4'].predict(X_poly))
        full_test_data['predicted_updrs_4'] = full_test_data['predicted_updrs_4'].apply(lambda x: 0 if x < 0 else x)"""
        
        """poly = PolynomialFeatures(degree = 5)
        X_poly = poly.fit_transform(X)
        full_test_data['predicted_updrs_1'] = np.ceil(model_dict['updrs_1'].predict(X_poly))"""
        
        ## for all other models
        ##full_test_data['result_' + str(u)] = np.ceil(model_dict[u].predict(X))
        
        for m in [0, 6, 12, 24]:
            for u in [1, 2, 3, 4]:
                temp = full_test_data[['visit_id', 'visit_month', 'predicted_updrs_' + str(u) + '_plus_' + str(m) + '_months']]
                temp['prediction_id'] = temp['visit_id'] + '_updrs_' + str(u) + '_plus_' + str(m) + '_months'
                
                #if (u == 2) | (u == 4):
                if (u == 4):
                    temp["rating"] = 0
                else:
                    #print('prediction is :',temp["predicted_updrs_" + str(u)].apply(lambda x: 0 if x < 0 else x))
                    temp["rating"] = temp["predicted_updrs_" + str(u) + '_plus_' + str(m) + '_months'].apply(lambda x: 0 if x < 0 else x)
                
                temp = temp[['prediction_id', 'rating']]

                result = result.append(temp)
                
        result = result.drop_duplicates(subset=['prediction_id', 'rating'])
        result = result.reset_index()
        result.drop(columns=['index'], inplace=True)
    
    return result

In [18]:
"""pred_vs_true_melt = pd.melt(temp, id_vars=['visit_id'], value_vars=['updrs_1','updrs_2','updrs_3','updrs_4','predicted_updrs_1','predicted_updrs_2','predicted_updrs_3'\
                                                                    ,'predicted_updrs_4'])

pred_vs_true_melt.head(10)

y_true_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['updrs_1','updrs_2','updrs_3','updrs_4'])]['value']
y_pred_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['predicted_updrs_1','predicted_updrs_2','predicted_updrs_3','predicted_updrs_4'])]['value']

print('SMAPE + 1 on overall test data :', smape_plus_1(np.array(y_true_overall), np.array(y_pred_overall)))"""

"pred_vs_true_melt = pd.melt(temp, id_vars=['visit_id'], value_vars=['updrs_1','updrs_2','updrs_3','updrs_4','predicted_updrs_1','predicted_updrs_2','predicted_updrs_3'                                                                    ,'predicted_updrs_4'])\n\npred_vs_true_melt.head(10)\n\ny_true_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['updrs_1','updrs_2','updrs_3','updrs_4'])]['value']\ny_pred_overall = pred_vs_true_melt[pred_vs_true_melt['variable'].isin(['predicted_updrs_1','predicted_updrs_2','predicted_updrs_3','predicted_updrs_4'])]['value']\n\nprint('SMAPE + 1 on overall test data :', smape_plus_1(np.array(y_true_overall), np.array(y_pred_overall)))"

In [19]:
test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')

test_proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')

test_peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')

#result = get_predictions(model_dict=random_forest_model_dict, df_test=test, df_proteins=test_proteins, df_peptides=test_peptides)
#result = get_predictions_v2(model_dict=ridge_model_dict, df_test=test)
#result = get_predictions(model_dict=random_forest_model_dict, df_test=test, df_proteins=test_proteins)
result = get_predictions_v4(model_dict=ridge_model_dict, df_test=test, df_peptides=test_peptides)

#result = get_predictions(model_dict=model_dict, sklearn_pipeline=full_pipeline)

result

no proteins only peptides
df_test shape and number of unique visit_ids: (16, 6) 4
df_peptides shape and number of unique visit_ids: (2057, 7) 2
full_test_data shape and number of unique visit_ids: (4, 12) 4
  visit_id  visit_month  visit_month_lag  visit_month_lead             F  \
0   3342_0            0             -1.0               6.0  0.000000e+00   
1   3342_6            6              0.0              -1.0  1.160877e+07   
2  50423_0            0             -1.0               6.0  1.404003e+07   
3  50423_6            6              0.0              -1.0  0.000000e+00   

              Y             W             A             L             V  \
0  0.000000e+00       0.00000  0.000000e+00  0.000000e+00  0.000000e+00   
1  7.815853e+06  478501.27662  5.481062e+07  7.032429e+07  3.819463e+07   
2  9.267859e+06  503543.34725  6.413289e+07  8.498561e+07  4.585628e+07   
3  0.000000e+00       0.00000  0.000000e+00  0.000000e+00  0.000000e+00   

              I  
0  0.000000e+00  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,prediction_id,rating
0,3342_0_updrs_1_plus_0_months,6.0
1,3342_6_updrs_1_plus_0_months,8.0
2,50423_0_updrs_1_plus_0_months,6.0
3,50423_6_updrs_1_plus_0_months,8.0
4,3342_0_updrs_2_plus_0_months,5.0
...,...,...
59,50423_6_updrs_3_plus_24_months,17.0
60,3342_0_updrs_4_plus_24_months,0.0
61,3342_6_updrs_4_plus_24_months,0.0
62,50423_0_updrs_4_plus_24_months,0.0


In [20]:
import amp_pd_peptide

env = amp_pd_peptide.make_env()   # initialize the environment for one run only

"""amp_pd_peptide.make_env.__called__ = False
type(env)._state = type(type(env)._state).__dict__['INIT']"""

iter_test = env.iter_test()

for (test, test_peptides, test_proteins, submission) in iter_test:
    #submission = get_predictions_v2(model_dict=ridge_model_dict, df_test=test)
    submission = get_predictions_v4(model_dict=ridge_model_dict, df_test=test, df_peptides=test_peptides)
    #submission = get_predictions(model_dict=decision_model_dict, df_test=test, df_proteins=test_proteins, df_peptides=test_peptides)
    env.predict(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
no proteins only peptides
df_test shape and number of unique visit_ids: (8, 5) 2
df_peptides shape and number of unique visit_ids: (1021, 6) 1
full_test_data shape and number of unique visit_ids: (2, 12) 2
  visit_id  visit_month  visit_month_lag  visit_month_lead             F  \
0   3342_0            0             -1.0              -1.0  0.000000e+00   
1  50423_0            0             -1.0              -1.0  1.404003e+07   

              Y             W             A             L             V  \
0  0.000000e+00       0.00000  0.000000e+00  0.000000e+00  0.000000e+00   
1  9.267859e+06  503543.34725  6.413289e+07  8.498561e+07  4.585628e+07   

              I  
0  0.000000e+00  
1  1.172552e+07  
no proteins only peptides
df_test shape and number of unique visit_ids: (8, 5) 2
df_peptides shape and number of unique visit_ids: (1036, 6) 1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

full_test_data shape and number of unique visit_ids: (2, 12) 2
  visit_id  visit_month  visit_month_lag  visit_month_lead             F  \
0   3342_6            6             -1.0              -1.0  1.160877e+07   
1  50423_6            6             -1.0              -1.0  0.000000e+00   

              Y             W             A             L             V  \
0  7.815853e+06  478501.27662  5.481062e+07  7.032429e+07  3.819463e+07   
1  0.000000e+00       0.00000  0.000000e+00  0.000000e+00  0.000000e+00   

              I  
0  1.045155e+07  
1  0.000000e+00  
